In [1]:
from importlib import reload
import sys
sys.path.insert(1, '../scripts')
from solve import *
import numpy as np
import imgkit

In [2]:
from IPython.display import display, HTML, clear_output, Javascript
import time
import math
from geographiclib.geodesic import Geodesic
geod = Geodesic.WGS84
from ipyleaflet import Map, Marker, Icon, Polyline, WidgetControl, CircleMarker
from ipywidgets import HTML as cHTML
from ipywidgets import IntProgress, Label, HBox
import pandas as pd
import imgkit
import os
from PIL import Image
import datetime
from ipywidgets.embed import embed_minimal_html, dependency_state
import glob
%matplotlib notebook
import matplotlib.pyplot as plt
import random

In [3]:
source = 'R2R'

In [4]:
game_data, arcs_data, travel_data = prep_data('diesel')

t_d = pd.read_pickle('../data/travel_info.p')
t_d = t_d.set_index(['city_x', 'city_y'])

venue_data = t_d[['info_x', 'venue_x']].reset_index(level=1, drop=True).drop_duplicates(subset='venue_x')
venue_data.index.name = 'city'
venue_data.columns = ['info', 'venue']
def split_coors(r):
    r['latitude'] = r['info'].latitude
    r['longitude'] = r['info'].longitude
    r['address'] = r['info'].address
    return r
venue_data = venue_data.apply(lambda i: split_coors(i), axis=1).drop('info', axis=1)


In [5]:
my_icon = 'data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAACgAAAA0CAYAAAD46nqNAAAACXBIWXMAAC4jAAAuIwF4pT92AAALkklEQVR42u1Ze1RU1Ro/MFfE4vJSQUwRCBPvBeWRCD6WCpHIzdRA8EWW8jSYYWZ4GsNjhvdDLbu3rlmr64ubZV7NxFIrK3OppQKCCKllYr6KsgTmcfZ3v2/DELoUB6W1+sOz1l5n5sw+e//27/t9j71HyMvLE/7MTXgA8AHA27Tc3NzuO7WcnBwzVWczNzZ6Zvy95zt/GEDjZMamUqkk+FySn58vFJeUCOXl5UJZWRm2Un4vLi4W8jvfNae+t77frwB7soDMmGs0GqGiokLIzcsXVNkvDogIf8Zmyozg0UGhYROCw57ynxY6yzNy0UKbnJxcizy1WqhA8Bq807t9ZdTklSALZmpkq6ysXEiWycyfCAmZ5zLaY63DGM/mYT6TfnGbHd3utSxNHBOZIDo/OV9n7zHuF+thI845u7q/HhI662mZXCkhZmkMMr+pIE1ij/RFg6dlZgkT/AOW2I50O+6xIBHmv30Q4g5dAdkpETIuAKy8ApB5ESDtW4Dk2naIrm6EJys2gNOUWWBpY3/MPyBgfjqOUVhYyBd8Xwz20Ao36eLoZ0c4uj6602NREjz/8VnIaAEx7TzoFd+AqGwGlnLKwGQNen6XNzKmwGfKcyBmXAR9Kt4Xbj8Cw6eGgZ394O2pmVlDi4qKuk3eG5O9ao5WWVhQKEybNn2kYCZpCi56g1jSKc6AKK0nMCKTnwEWU6dnifUirKjrgLhaHcTV6SHhFDAEz6QndUzaYGDK70BM/x70UzIr4WE7+7qgoKARBNLI5J1A3hFcdnY2eaNZ5MJF9sIAy9NzXt0JGT9ARzJNWK8HOTKU2AQQs/c0qKp2Q/E71RAufxE2HaqDVz/6EhTbPoOYo2j+MwApjSJ/h4BmXYaOsLXvgcRyUI1cqbRVq9VmvTnObVEj9eR1ZqnpGcJge/tNgcoSWHkVdMl1WtSbAVKQtec/vwhl1V9C85mzwC9RD82n6js/MwO0X2+FnZ8ehKVbPsb+DFKaGErAAMnIctYV0AakFIGD47A3i4pLhLzcXLM+MYgAJUUo5PE+Pk9YuXiAtO43vey0nrOgOAsselctVO3eC/q23zieS1euwMaNm+C1f6+D67/+Cj0vdV4uPDrnWXQcPcgacQxcoKzRIMqbDHoHn8kQPHNmsKag4KYQdFeAGHzNcvBuaWVdHbb2XTHtAuiTSXOnGVtxUgeK1es4SyIC2LNnD0ycOBFsbW1hyJAh4OfnB1u2bIHa2lrYt38/JCQmgiAIELa6CtCpuJmT63Us/SLoUNPMwsb2gzwEeCcWbxPvcngWiIiMesTG1aP9hRM3GJqHnALSWwDCKjZBVqoCDKIIjDEIDQ3lACwtLcHCwoJ/jo2NhZaWFg7yo7174aFBg8Bq+CiIx5CE2gVikcZM/KqV2biOaQufH+FEc6LD3B1gtipHUoaRf8q06TEuTy2G9AtgSG7QiymNGEaadODs7Q+HPj8AjKSGANetW8dBGZu5uTkcPnz4JjOnpafz3+b+awdfJFkDmSSvNowMiYAp04OWUALIVqnMTWBQ9RcCOGmivzpw5Uss6xLTvlDbzigAP7n2f5CZlson1en1ICKLra2tEBUVBQEBAeDu7s7v169f57/psY/BYACtTgfjxo4B/8UJkP49IzOD9KQWPZppx8dmsQA/H3V5RSWf++4As7MH0GomBUwsCFnzLgVkXRJmBRwMphW9BalJiXzStrY2DvT48eNw7dq1TudFRvft2wf19Z3erNVqed/Wn3+G+hPHILPiZYyTWjIvkGRw0brpJRsgcMLjBUSKSpU9wCQGS0vLhcmBARoCSIMQgxgDYdaat2FVoRquXPuRAyBg69evhxs3bnDGCCCxt3nzZv7MeO2u3gMisphZvhZiEaAc4yfKhmGW0c0o2wRoLQ2lUtMYJBNjZ2IweNVWhgxqk+o6WBqaZt6GT2Dnjh0glcogPj4esG83cwSQGl0NDQ2wevVqOHDgAPfojQiYrqL39kPcSQPIT3MGWeYPoJ2S9xoE+PkWULmG1jMRYHkFafDFCYpS0omOskfKacwaNR1QsWErxCxdwkVvbW0NTU1NcOtVU1MDg9BzJRIJ7/f5wYPQeukiJO06BjKM61Rc0JhZV5jOX1nKJgdMzOk0sQkMYoozJ8FO8PWZ6RwSwbBKESl20aArGkXI/uIshM+d0+21gYGBcPXqVW5aapcvXwZvb2/+mzHsvPXmG7D70New5OPzgPkZw4zI4yF6sThq1gI2eerU2cSgqV7MK+Sn/xFma+3y2I2kmjYUNYLEwgBzKoI0wLLif4LXY+7dIBUKBVRWVgIWsWh+6U1hJzhoBnzyYTWk7DsFSc2M8jJmEsZoTEqdNm5j2+bMnm1Pc5oUB7sTd75aeMja7sM566tJf3ryOmJRjgzEfHYeXqraDmFB02FFQvxN5tWjDitRfy5ubqCUJsGRI0chfdsBWH7sOiiaOTgeZlK/Y/rIrYeY1eChu3Ny80wvFroKVAklce/x4xM8ouIBxcxTHa2cQMowGySeuAEZ2/bBxq3b4PAXB3jqo4KBe/fFC1C1eRO8/+VXEPtBDTJnQHCkPcbZo7EwKui9lqWCr69PPM8iOKfJ1QxRXVpaKsydN8/ewnHEj4lHf2LybxiZGchZiAVZI96xcl66/xw8+8p/YVdLO7xXew52nWsF2X/eh5iqTzDm/QrJ6ENy3p/xd/kYmOZeOH6dPeTs/hPOMZjmup1578ggb1hdqDUFwtChQ1+asnINj1nJJ7tYPE1aYtzkWLxyz0xoYN1NijJIRsYUTdDZj4MzsoeFQgvTha55BxyHP/KaGqv13Fu2qCaV/LSxKeksGv5mO9ZXm9KoFWWNIrVuFjvvIgchx8/GRt9TerDW846LEpVnRNFpUog2PGK+Z0lJcfcmqs8lP61MU1gkDHdxq5pR8AYleh0FWGSQGSc1pXVKglE9ybD40M1ctRWcnF03q7HMyr3LDq/XTRNnEd0/ImrBWCuXMb8lHf/FkNIk8pBDGuwbQHrHIMrq2wz2nv7t4VFRXjS28QTinradXVrkLDo4OL4cgKV/l0fTxCax2NWHV+O0w5uhXgdOI0e9QmPm3gWcSftiVacWzZ6ZH+kksXO4lnDokqg4wyi7mMwgFajyb0RxBUYDjArX5kdGOlJo6U17pjPYddxRgHsUbz8/qfvsJVjToRYp/VFG6I1Fo/YoraH2Hpu3DLx9fZNpLFP2xCYffdAur0CjMVdmZA0Y+Fe7I3Ne3wNpuMeV1hvuaOqepqUN/tw394KFlfVBZXqGGY1FY/bL2czvm/gcMzLLpKlBk2zcPbXSuht6OTqMrCt431Z/9Fsz9qlv19uM9up43N8/oCvn9t/ZzM2mVpkXY9Qf4uBY5LVUwattZLEzePdg0RhWKKVRaPJcLIXBQ4YUFJeUmmzaezrANJp6WWzsQEsbu68jNnwKqedxU9XDq42mxawDuF01zF3/IQx82OrI8pi4gZo+mPaeDjC78rQ5nQ0uWLjIx3KEW0fSiZ8N5KHSHqaWNmAKPMvEuC++N0jsHTsWRkePL8ciWNVV7/X7AebtvFpdUCj8fdx46djIOHICNLXOmGGY9JSepZ4TdS5PzANPbx+ZusfJQV+Pgu/p3LjL1JL0rJVm5hYDd8xavRUyL3UGcGr0eVJqOZhbDtqRiaxp1GpJX017zwB7ejWCFJ5bHuto/YjL+UU7vmLKb8GgPMsMS94/AdZOzt8tT0waQn1UfThR7Z+/IX4/opOsWrVKCJw8ebrEbphhxdGrekWTQf/w6HGGqUFBMypRq9nZ/LD9jz/l702PdFA+eoyHdBRqziM8Bjy8xsvykbl71V2//pGT0zm5hOLjMGfX7cNdH32H4h09ux9g/QKwhx6F4qJCIfq5eCE6NkEopkPyLqe4X5D3vUIjAPLSwkKNUFjATdsv4Prtv7qeIPsT3IN/Ox8A/DO0/wP+ieaxV3jMgAAAAABJRU5ErkJggg=='

In [6]:
global imgcnt
imgcnt = 0

def print_current_solution(given_result, name):
    global imgcnt
    imgcnt = 0
    center=(55, 22)
    from ipyleaflet import Map
    m = Map(center=center, zoom=4, min_zoom=1, zoom_control=False)
    m.layout.height = '600px'

    time.sleep(0.5)
    display(m)

    label = Label()
    prog = IntProgress(value=0, max=len(given_result))
    prog.layout.width='250px'
    h = HBox([label, prog])
    m.add_control(WidgetControl(widget=h, position='topright'))

    def add_marker(m, i, r):
        global imgcnt
        icon = Icon(icon_anchor=(20,44), icon_size=(40,52), icon_url=my_icon)
        marker = Marker(location=(i.latitude, i.longitude), icon=icon)
        m.add_layer(marker)

    def fly_between(m, i, j):
        t = geod.InverseLine(i.latitude, i.longitude, j.latitude, j.longitude)
        ds = 1e5
        #ds = 1e20
        n = int(math.ceil(t.s13 / ds))
        current_point = None
        for i in range(n + 1):
            s = min(ds * i, t.s13)
            a = t.Position(s)
            
            time.sleep(0.01)
                
            if current_point:
                piece = Polyline(locations=[list(current_point), [a['lat2'], a['lon2']]], color='blue', weight=2)
                m.add_layer(piece)                                        
            current_point = (a['lat2'], a['lon2'])

    def drive_between(m, i, j, color='green'):
        pr = 15
        diff = [(j.latitude - i.latitude)/pr, (j.longitude - i.longitude)/pr]
        previous_point = [i.latitude, i.longitude]
        for _ in range(pr):
            current_point = [x+y for x, y in zip(diff, previous_point)]
            piece = Polyline(locations=[previous_point, current_point], color=color, weight=2)
            m.add_layer(piece)
            previous_point = current_point
            time.sleep(0.1/pr)

    step = 0
    markers_to_remove = []
    for c, r in enumerate(given_result.itertuples()):
        for i in markers_to_remove:
            m.remove_layer(i)
        markers_to_remove = []
        label.value = str(r.start_dt)
        prog.value = step + 1
        step += 1
        if r.type == 'Game':
            # mark other games
            other_games = game_data[game_data['date'] == pd.to_datetime(r.start_date) + datetime.timedelta(days=1)]
            # mark selected game
            i = venue_data.loc[r.location]
            add_marker(m, i, r)
        elif r.type == 'Flight':
            i = venue_data.loc[r.location]
            j = venue_data.loc[r.destination]
            fly_between(m, i, j)
        elif r.type == 'Bus':
            i = venue_data.loc[r.location]
            j = venue_data.loc[r.destination]
            drive_between(m, i, j, color='black')
        elif r.type == 'Train':
            i = venue_data.loc[r.location]
            j = venue_data.loc[r.destination]
            drive_between(m, i, j, color='brown')
        else:
            i = venue_data.loc[r.location]
            j = venue_data.loc[r.destination]
            drive_between(m, i, j)
        time.sleep(0.3)
    return m

In [7]:
def round_up(n, decimals=2): 
    multiplier = 10 ** decimals
    return math.ceil(n * multiplier) / multiplier    


def fix_headers_currency(tb2, emission=False):
    tb3 = tb2.copy()
    if emission:
        tb3.columns = ['Activity', 'Detail', 'Location', 'Destination', 'Price (€)', 'Emission (kg)', 'Start', 'Finish']
    else:
        tb3.columns = ['Activity', 'Detail', 'Location', 'Destination', 'Price (€)', 'Start', 'Finish']
    tb4 = tb3.copy()
    try:
        tb4['Price (€)'] = round(tb3['Price (€)'] / 1.101, 2)
    except:
        pass
    tb5 = tb4.copy()
    try:
        tb5.loc[tb5['Price (€)'] <= 1, 'Price (€)'] = ''
    except:
        pass
    if emission:
        tb5['Emission (kg)'] = round(tb5['Emission (kg)'] / 1000, 2)
        try:
            tb5.loc[tb5['Emission (kg)'] < 0.01, 'Emission (kg)'] = ''
        except:
            pass
    
    def newlineid(x):
        if 'group' in x:
            return x.split(' [ID')[0]
        else:
            return x
    tb5['Detail'] = tb5['Detail'].apply(lambda x: newlineid(x))
    
    return tb5

In [8]:
e1 = e = pd.read_csv(f'../results/{source}_all_method_shortest_tour_d.csv', index_col=0)
map1 = print_current_solution(e, name='Shortest Tour')
tb = e[['type', 'name', 'location', 'destination', 'price', 'start_dt', 'finish_dt']].replace(np.nan, '', regex=True).copy()
tb['name'] = tb['name'].apply(lambda i: i.split('(')[0])
tb = fix_headers_currency(tb.copy())
HTML(fix_headers_currency(tb).to_html())

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

,Activity,Detail,Location,Destination,Price (€),Start,Finish
0,Game,France vs Germany,"Munich, Germany",,,2020-06-16 21:00:00,2020-06-16 22:45:00
1,Flight,Fly from Munich,"Munich, Germany","Baku, Azerbaijan",394.64,2020-06-17 03:18:30,2020-06-17 13:26:30
2,Game,Turkey vs Wales,"Baku, Azerbaijan",,,2020-06-17 18:00:00,2020-06-17 19:45:00
3,Flight,Fly,"Baku, Azerbaijan","Bucharest, Romania",270.21,2020-06-18 00:18:00,2020-06-18 10:27:00
4,Game,Ukraine vs [Playoff-D/A],"Bucharest, Romania",,,2020-06-18 15:00:00,2020-06-18 16:45:00
5,Flight,Fly to Glasgow,"Bucharest, Romania","Glasgow, Scotland",118.07,2020-06-19 01:51:30,2020-06-19 08:53:30
6,Game,Croatia vs Czech Republic,"Glasgow, Scotland",,,2020-06-19 18:00:00,2020-06-19 19:45:00
7,Flight,Fly from Glasgow,"Glasgow, Scotland","Budapest, Hungary",142.6,2020-06-20 02:27:30,2020-06-20 08:17:30
8,Game,[Playoff-A/D] vs France,"Budapest, Hungary",,,2020-06-20 15:00:00,2020-06-20 16:45:00
9,Flight,Fly to Rome,"Budapest, Hungary","Rome, Italy",119.89,2020-06-21 02:53:30,2020-06-21 07:51:30


In [9]:
map1

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [10]:
e2 = e = pd.read_csv(f'../results/{source}_all_method_least_expensive_d.csv')
map2 = print_current_solution(e, "Shortest Travel Time")
tb = e[['type', 'name', 'location', 'destination', 'price', 'start_dt', 'finish_dt']].replace(np.nan, '', regex=True)
tb['name'] = tb['name'].apply(lambda i: i.split('(')[0])
tb = fix_headers_currency(tb.copy())
HTML(tb.to_html())

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

,Activity,Detail,Location,Destination,Price (€),Start,Finish
0,Game,Belgium vs Russia,"Saint Petersburg, Russia",,,2020-06-13 21:00:00,2020-06-13 22:45:00
1,Flight,"Fly Pulkovo to London Gatwick, train","Saint Petersburg, Russia","London, England",177.11,2020-06-14 03:25:00,2020-06-14 10:20:00
2,Game,England vs Croatia,"London, England",,,2020-06-14 15:00:00,2020-06-14 16:45:00
3,Bus,Bus and ferry,"London, England","Dublin, Ireland",16.8,2020-06-14 23:07:30,2020-06-15 11:37:30
4,Game,Poland vs [Playoff-B],"Dublin, Ireland",,,2020-06-15 18:00:00,2020-06-15 19:45:00
5,Bus,"Bus and ferry, bus","Dublin, Ireland","Copenhagen, Denmark",89.92,2020-06-16 12:51:30,2020-06-18 00:53:30
6,Game,Denmark vs Belgium,"Copenhagen, Denmark",,,2020-06-18 18:00:00,2020-06-18 19:45:00
7,Bus,Bus,"Copenhagen, Denmark","Munich, Germany",56.77,2020-06-19 10:25:00,2020-06-20 03:20:00
8,Game,Portugal vs Germany,"Munich, Germany",,,2020-06-20 18:00:00,2020-06-20 19:45:00
9,Bus,Bus,"Munich, Germany","Rome, Italy",47.68,2020-06-21 00:42:30,2020-06-21 13:02:30


In [11]:
map2

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [12]:
e3 = e = pd.read_csv(f'../results/{source}_all_method_least_emissiondiesel_d.csv')
map3 = print_current_solution(e, "Least Emission")
tb = e[['type', 'name', 'location', 'destination', 'price', 'emission', 'start_dt', 'finish_dt']].replace(np.nan, '', regex=True)
tb['emission'] = tb['emission'].round(2)
tb['name'] = tb['name'].apply(lambda i: i.split('(')[0])
tb = fix_headers_currency(tb.copy(), emission=True)
HTML(tb.to_html())

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

,Activity,Detail,Location,Destination,Price (€),Emission (kg),Start,Finish
0,Game,Belgium vs Russia,"Saint Petersburg, Russia",,,,2020-06-13 21:00:00,2020-06-13 22:45:00
1,Flight,"Train to Helsinki, fly to Edinburgh, bus","Saint Petersburg, Russia","Glasgow, Scotland",210.72,249.87,2020-06-14 14:03:30,2020-06-14 23:41:30
2,Game,[Playoff-C] vs Czech Republic,"Glasgow, Scotland",,,,2020-06-15 15:00:00,2020-06-15 16:45:00
3,Train,Train via Frankfurt Airport,"Glasgow, Scotland","Munich, Germany",770.21,10.85,2020-06-15 23:03:30,2020-06-16 14:41:30
4,Game,France vs Germany,"Munich, Germany",,,,2020-06-16 21:00:00,2020-06-16 22:45:00
5,Train,Train,"Munich, Germany","Rome, Italy",117.62,5.45,2020-06-17 05:09:00,2020-06-17 14:36:00
6,Game,Italy vs Switzerland,"Rome, Italy",,,,2020-06-17 21:00:00,2020-06-17 22:45:00
7,Train,"Train, car ferry","Rome, Italy","Dublin, Ireland",722.98,26.11,2020-06-18 06:29:30,2020-06-19 07:15:30
8,Game,Sweden vs [Playoff-B],"Dublin, Ireland",,,,2020-06-19 15:00:00,2020-06-19 16:45:00
9,Train,"Car ferry, train","Dublin, Ireland","Copenhagen, Denmark",716.17,24.16,2020-06-20 18:38:00,2020-06-21 19:07:00


In [13]:
map3

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [14]:
e4 = e = pd.read_csv(f'../results/{source}_all_method_least_emissionelectric_d.csv')
map4 = print_current_solution(e, "Least Emission (Electric)")
tb = e[['type', 'name', 'location', 'destination', 'price', 'emission', 'start_dt', 'finish_dt']].replace(np.nan, '', regex=True)
tb['emission'] = tb['emission'].round(2)
tb['name'] = tb['name'].apply(lambda i: i.split('(')[0])
tb = fix_headers_currency(tb.copy(), emission=True)
HTML(tb.to_html())

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

,Activity,Detail,Location,Destination,Price (€),Emission (kg),Start,Finish
0,Game,Wales vs Switzerland,"Baku, Azerbaijan",,,,2020-06-13 15:00:00,2020-06-13 16:45:00
1,Drive,Drive,"Baku, Azerbaijan","Budapest, Hungary",449.59,10.13,2020-06-14 07:55:30,2020-06-16 02:49:30
2,Game,[Playoff-A/D] vs Portugal,"Budapest, Hungary",,,,2020-06-16 18:00:00,2020-06-16 19:45:00
3,Drive,Drive,"Budapest, Hungary","Bucharest, Romania",104.45,2.42,2020-06-17 12:36:30,2020-06-17 22:08:30
4,Game,Ukraine vs [Playoff-D/A],"Bucharest, Romania",,,,2020-06-18 15:00:00,2020-06-18 16:45:00
5,Drive,Drive,"Bucharest, Romania","Munich, Germany",217.98,4.29,2020-06-19 09:29:30,2020-06-20 01:15:30
6,Game,Portugal vs Germany,"Munich, Germany",,,,2020-06-20 18:00:00,2020-06-20 19:45:00
7,Train,"Car train, drive","Munich, Germany","Rome, Italy",235.24,2.39,2020-06-21 02:22:30,2020-06-21 11:22:30
8,Game,Italy vs Wales,"Rome, Italy",,,,2020-06-21 18:00:00,2020-06-21 19:45:00
9,Drive,Drive,"Rome, Italy","Copenhagen, Denmark",358.76,6.15,2020-06-21 22:31:30,2020-06-22 18:13:30


In [15]:
map4

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [16]:
elems = []

for j, i in enumerate([e1, e2, e3, e4]):
    v = i['emission'].sum() / 1000
    fn = i['finish_dt'].iloc[-1]
    st = i['start_dt'].iloc[0]
    d = datetime.datetime.strptime(fn, '%Y-%m-%d %H:%M:%S') - datetime.datetime.strptime(st, '%Y-%m-%d %H:%M:%S')
    i['dt_diff'] = i.apply(lambda k: datetime.datetime.strptime(k.finish_dt, '%Y-%m-%d %H:%M:%S') - datetime.datetime.strptime(k.start_dt, '%Y-%m-%d %H:%M:%S'), axis=1)
    i.loc[i.type == 'Game', 'dt_diff'] = pd.Timedelta(seconds=0)
    total_travel = i['dt_diff'].sum()
    cost = i['price'].sum()
    cost = round(cost/1.101,2)
    elems.append({'solution': j, 'emission': v, 'duration': d, 'cost': cost})
mydf = pd.DataFrame(elems)

mydf['solution'] = ['Shortest Tour', 'Least Expensive', 'Least Emission (Diesel)', 'Least Emission (Electric)']

pcopy = mydf[['solution', 'duration', 'cost', 'emission']].copy()
pcopy['emission'] = pcopy['emission'].round(2)
pcopy.columns = ['Solution', 'Duration', 'Cost (€)', 'Emission (kg)']

HTML(pcopy.to_html())

,Solution,Duration,Cost (€),Emission (kg)
0,Shortest Tour,12 days 22:45:00,2217.98,3159.20
1,Least Expensive,20 days 22:45:00,1052.23,1720.91
2,Least Emission (Diesel),20 days 22:45:00,4127.61,592.16
3,Least Emission (Electric),20 days 04:45:00,3652.59,76.88
